In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import geopy.geocoders

from shapely.geometry import Point
from functools import partial
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import GoogleV3

### Methods Section

In [3]:
def addressGeocoded(df,gc):
    '''
    This function accepts adataframe and a geocode object and returns a geo dataframe
    
    :param df: dataframe with addresses
    :param gc: geocode object
    
    :returns geodataframe
    '''
    tmp_df = df.copy()
    tmp_df['geolocation'] = tmp_df['address'].apply(gc) #translate address into coordinate
    tmp_df['geometry'] = tmp_df['geolocation'].apply(lambda loc: tuple(loc.point[0:2]) if loc else None) #extract tuple point location
    tmp_df['geometry'] = tmp_df['geometry'].apply(Point) # convert tuple in shapely geometry point
    return tmp_df

### Load Datasets

In [4]:
### Load Douro wine producers
douro_wine_producer_df = pd.read_excel("./data_source/douro/douro_wine_producers_list.xlsx")
douro_wine_producer_df

,producer,address
0,Niepoort,"670 Rua Candido do Reis, Vila Nova de Gaia 4400"
1,Poeira,"Provesende 5060, Villa Real"
2,Quinta do Couquinho,"Horta da Vilarica, 5160-101 Torre de Moncorvo,..."


In [7]:
### Istance Google connection for geocoding
geopy.geocoders.options.default_user_agent = 'wine_app'
geopy.geocoders.options.default_scheme = 'https'
with open("./data_source/apikey.txt") as f:   #load Google API key
    apikey = f.readline()
    f.close
geolocator = GoogleV3(api_key=apikey)
geocode = partial(geolocator.geocode, language="en")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=5)
# print(geocode)

In [8]:
douro_wine_producer_gdf = addressGeocoded(douro_wine_producer_df,geocode)

In [10]:
douro_wine_producer_gdf

,producer,address,geolocation,geometry
0,Niepoort,"670 Rua Candido do Reis, Vila Nova de Gaia 4400","(R. Cândido dos Reis 670, 4400-071 Vila Nova d...",POINT (41.1310385 -8.6109353)
1,Poeira,"Provesende 5060, Villa Real","(R. Fundo de Vila 20, 5060-263 Provesende, Por...",POINT (41.2171499 -7.5666741)
2,Quinta do Couquinho,"Horta da Vilarica, 5160-101 Torre de Moncorvo,...","(5160-101, Portugal, (41.2319091, -7.122560699...",POINT (41.2319091 -7.122560699999999)
